<a href="https://colab.research.google.com/github/VarunB1234/Depression-Detection/blob/main/LSTM_Without_Gating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!pip install --upgrade numpy
!pip install --upgrade gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 113.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 114.2 MB/s eta 0:00:00
   ━━━━━━━━━━

In [1]:
import gc
import nltk
nltk.download('stopwords')
import math

from smart_open import open
from nltk.corpus import stopwords
import sklearn
from sklearn.metrics import classification_report
from keras.layers import Dropout
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors

import tensorflow as tf

from tensorflow import keras

from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras import layers


from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Text
class Highway(layers.Layer):

  def __init__(self):
    super(Highway, self).__init__()

  def build(self, input_shape):
    n_sentences = input_shape[1]
    n_features = input_shape[2]
    carry_bias = keras.initializers.Constant(value=-1.0)
    random_dist = keras.initializers.RandomNormal(mean=0.0, stddev=0.1)

    carry_bias_2 = keras.initializers.Constant(value= 0.1)

    self.W_T = self.add_weight(shape=(n_features, n_features),initializer = random_dist,trainable=True)
    self.b_T = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias, trainable=True)

    self.W = self.add_weight(shape=( n_features, n_features),initializer = random_dist, trainable=True)
    self.b = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias_2, trainable=True)


  def call(self, inputs):
    x = inputs
    T = tf.sigmoid(tf.matmul(x, self.W_T) + self.b_T, name="transform_gate")
    H = tf.nn.relu(tf.matmul(x, self.W) + self.b, name="activation")
    C = tf.subtract(1.0, T, name="carry_gate")

    return tf.add(tf.multiply(H, T), tf.multiply(x, C), "y")



#LSTM FOR TEXT
# first input model
class text:
  input3 = Input(shape = (250,5100))
  dense4 = Dense(1000)(input3)
  dense5 = Dense(500)(dense4)
  dense6 = Dense(250)(dense5)
  # interpretation model
  lstm = LSTM(128, dropout = 0.2, recurrent_dropout = 0.2)(dense6)
  output = Dense(1, activation='sigmoid')(lstm)
  model = Model(inputs=input3, outputs=output)
  # summarize layers
  # print(model.summary())
  # # plot graph
  # print(plot_model(model, to_file='multiple_inputs.png'))
  optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001)
  def run_model(self):
    self.model.compile(optimizer=self.optimizer, loss='binary_crossentropy')
    return self.model




In [3]:
#Audio

class Highway(layers.Layer):

  def __init__(self):
    super(Highway, self).__init__()

  def build(self, input_shape):
    n_sentences = input_shape[1]
    n_features = input_shape[2]
    carry_bias = keras.initializers.Constant(value=-1.0)
    random_dist = keras.initializers.RandomNormal(mean=0.0, stddev=0.1)

    carry_bias_2 = keras.initializers.Constant(value= 0.1)

    self.W_T = self.add_weight(shape=(n_features, n_features),initializer = random_dist,trainable=True)
    self.b_T = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias, trainable=True)
    # self.W_T = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1),trainable=True)
    # self.b_T = tf.Variable(tf.constant(-1.0, shape=[n_data_points, n_sentences, n_features]),trainable=True)

    self.W = self.add_weight(shape=( n_features, n_features),initializer = random_dist, trainable=True)
    self.b = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias_2, trainable=True)
    # self.W = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1), trainable=True)
    # self.b = tf.Variable(tf.constant(0.1, shape=[n_data_points, n_sentences, n_features]), trainable=True)


  def call(self, inputs):
    x = inputs
    T = tf.sigmoid(tf.matmul(x, self.W_T) + self.b_T, name="transform_gate")
    H = tf.nn.relu(tf.matmul(x, self.W) + self.b, name="activation")
    C = tf.subtract(1.0, T, name="carry_gate")

    return tf.add(tf.multiply(H, T), tf.multiply(x, C), "y")


#LSTM FOR AUDIO
# first input model

class audio:
  input3 = Input(shape = (250,74))
  # dense4 = Dense(1000)(input3)
  # dense5 = Dense(500)(dense4)
  # dense6 = Dense(250)(dense5)
  # interpretation model
  lstm = LSTM(60, dropout = 0.2, recurrent_dropout = 0.2)(input3)
  output = Dense(1, activation='sigmoid')(lstm)
  model = Model(inputs=input3, outputs=output)
  # summarize layers
  # print(model.summary())
  # plot graph
  # print(plot_model(model, to_file='multiple_inputs.png'))
  optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001)
  def run_model(self):
    self.model.compile(optimizer=self.optimizer, loss='binary_crossentropy')
    return self.model

In [4]:
#Video

class Highway(layers.Layer):

  def __init__(self):
    super(Highway, self).__init__()

  def build(self, input_shape):
    n_sentences = input_shape[1]
    n_features = input_shape[2]
    carry_bias = keras.initializers.Constant(value=-1.0)
    random_dist = keras.initializers.RandomNormal(mean=0.0, stddev=0.1)

    carry_bias_2 = keras.initializers.Constant(value= 0.1)

    self.W_T = self.add_weight(shape=(n_features, n_features),initializer = random_dist,trainable=True)
    self.b_T = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias, trainable=True)
    # self.W_T = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1),trainable=True)
    # self.b_T = tf.Variable(tf.constant(-1.0, shape=[n_data_points, n_sentences, n_features]),trainable=True)

    self.W = self.add_weight(shape=( n_features, n_features),initializer = random_dist, trainable=True)
    self.b = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias_2, trainable=True)
    # self.W = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1), trainable=True)
    # self.b = tf.Variable(tf.constant(0.1, shape=[n_data_points, n_sentences, n_features]), trainable=True)


  def call(self, inputs):
    x = inputs
    T = tf.sigmoid(tf.matmul(x, self.W_T) + self.b_T, name="transform_gate")
    H = tf.nn.relu(tf.matmul(x, self.W) + self.b, name="activation")
    C = tf.subtract(1.0, T, name="carry_gate")

    return tf.add(tf.multiply(H, T), tf.multiply(x, C), "y")


#LSTM FOR VIDEO
# first input model
class video:
  input3 = Input(shape = (250,388))
  # dense4 = Dense(1000)(input3)
  # dense5 = Dense(500)(dense4)
  dense6 = Dense(250)(input3)
  # interpretation model
  lstm = LSTM(128, dropout = 0.2, recurrent_dropout = 0.2)(dense6)
  output = Dense(1, activation='sigmoid')(lstm)
  model = Model(inputs=input3, outputs=output)
  # summarize layers
  print(model.summary())
  # plot graph
  print(plot_model(model, to_file='multiple_inputs.png'))
  optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001)
  def run_model(self):
    self.model.compile(optimizer=self.optimizer, loss='binary_crossentropy')
    return self.model

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 250, 388)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 250, 250)       │        97,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │       194,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 291,427 (1.11 MB)

 Trainable params: 291,427 (1.11 MB)

 Non-trainable params: 0 (0.00 B)

None
<IPython.core.display.Image object>


In [5]:
#Text + Audio
class Highway(layers.Layer):

  def __init__(self):
    super(Highway, self).__init__()

  def build(self, input_shape):
    n_sentences = input_shape[1]
    n_features = input_shape[2]
    carry_bias = keras.initializers.Constant(value=-1.0)
    random_dist = keras.initializers.RandomNormal(mean=0.0, stddev=0.1)

    carry_bias_2 = keras.initializers.Constant(value= 0.1)

    self.W_T = self.add_weight(shape=(n_features, n_features),initializer = random_dist,trainable=True)
    self.b_T = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias, trainable=True)
    # self.W_T = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1),trainable=True)
    # self.b_T = tf.Variable(tf.constant(-1.0, shape=[n_data_points, n_sentences, n_features]),trainable=True)

    self.W = self.add_weight(shape=( n_features, n_features),initializer = random_dist, trainable=True)
    self.b = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias_2, trainable=True)
    # self.W = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1), trainable=True)
    # self.b = tf.Variable(tf.constant(0.1, shape=[n_data_points, n_sentences, n_features]), trainable=True)


  def call(self, inputs):
    x = inputs
    T = tf.sigmoid(tf.matmul(x, self.W_T) + self.b_T, name="transform_gate")
    H = tf.nn.relu(tf.matmul(x, self.W) + self.b, name="activation")
    C = tf.subtract(1.0, T, name="carry_gate")

    return tf.add(tf.multiply(H, T), tf.multiply(x, C), "y")

class text_audio:
# first input model
  input1 = Input(shape=(250,74), name = 'Audio_input')
  # highway6 = Highway()(highway5)
  dense1 = Dense(74)(input1)

  input3 = Input(shape = (250,5100), name = 'Text_input')
  dense4 = Dense(1000)(input3)
  dense5 = Dense(500)(dense4)
  dense6 = Dense(250)(dense5)
  dense3 = Dense(74)(dense6)
  # merge input models
  merge = concatenate([dense1,dense3], axis = 1)
  # interpretation model
  lstm = LSTM(128, dropout = 0.2, recurrent_dropout = 0.2)(merge)
  output = Dense(1, activation='sigmoid')(lstm)
  model = Model(inputs=[input1, input3], outputs=output)
  # summarize layers
  # print(model.summary())
  # # plot graph
  # plot_model(model)
  optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001)

  def run_model(self):
    self.model.compile(optimizer=self.optimizer, loss='binary_crossentropy')
    return self.model

# inp = np.array([audio_train,video_train,text_train], dtype = object)


In [6]:
#Text+Video


class Highway(layers.Layer):

  def __init__(self):
    super(Highway, self).__init__()

  def build(self, input_shape):
    n_sentences = input_shape[1]
    n_features = input_shape[2]
    carry_bias = keras.initializers.Constant(value=-1.0)
    random_dist = keras.initializers.RandomNormal(mean=0.0, stddev=0.1)

    carry_bias_2 = keras.initializers.Constant(value= 0.1)

    self.W_T = self.add_weight(shape=(n_features, n_features),initializer = random_dist,trainable=True)
    self.b_T = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias, trainable=True)
    # self.W_T = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1),trainable=True)
    # self.b_T = tf.Variable(tf.constant(-1.0, shape=[n_data_points, n_sentences, n_features]),trainable=True)

    self.W = self.add_weight(shape=( n_features, n_features),initializer = random_dist, trainable=True)
    self.b = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias_2, trainable=True)
    # self.W = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1), trainable=True)
    # self.b = tf.Variable(tf.constant(0.1, shape=[n_data_points, n_sentences, n_features]), trainable=True)


  def call(self, inputs):
    x = inputs
    T = tf.sigmoid(tf.matmul(x, self.W_T) + self.b_T, name="transform_gate")
    H = tf.nn.relu(tf.matmul(x, self.W) + self.b, name="activation")
    C = tf.subtract(1.0, T, name="carry_gate")

    return tf.add(tf.multiply(H, T), tf.multiply(x, C), "y")

class text_video:
  input2 = Input(shape=(250,388), name = 'Video_input')
  # highway2 = Highway()(input2)
  # highway3 = Highway()(highway2)
  # highway4 = Highway()(highway3)
  # dense7 = Dense(200)(highway4)
  dense2 = Dense(250)(input2)

  input3 = Input(shape = (250,5100), name = 'Text_input')
  dense4 = Dense(1000)(input3)
  dense5 = Dense(500)(dense4)
  # dense6 = Dense(250)(dense5)
  dense3 = Dense(250)(dense5)
  # merge input models
  merge = concatenate([dense2,dense3], axis = 1)
  # interpretation model
  lstm = LSTM(128, dropout = 0.2, recurrent_dropout = 0.2)(merge)
  output = Dense(1, activation='sigmoid')(lstm)
  model = Model(inputs=[input2, input3], outputs=output)
  # summarize layers
  print(model.summary())
  # plot graph
  plot_model(model)
  optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001)

  def run_model(self):
    self.model.compile(optimizer= self.optimizer, loss='binary_crossentropy')
    return self.model

# inp = np.array([audio_train,video_train,text_train], dtype = object)

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Text_input          │ (None, 250, 5100) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 250, 1000) │  5,101,000 │ Text_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Video_input         │ (None, 250, 388)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 250, 500)  │    500,500 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 250, 250)  │     97,250 │ Video_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 250, 250)  │    125,250 │ dense_15[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 500, 250)  │          0 │ dense_13[0][0],   │
│ (Concatenate)       │                   │            │ dense_16[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ (None, 128)       │    194,048 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 1)         │        129 │ lstm_4[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,018,177 (22.96 MB)

 Trainable params: 6,018,177 (22.96 MB)

 Non-trainable params: 0 (0.00 B)

None


In [7]:
#Text+Audio+Video

class Highway(layers.Layer):

  def __init__(self):
    super(Highway, self).__init__()

  def build(self, input_shape):
    n_sentences = input_shape[1]
    n_features = input_shape[2]
    carry_bias = keras.initializers.Constant(value=-1.0)
    random_dist = keras.initializers.RandomNormal(mean=0.0, stddev=0.1)

    carry_bias_2 = keras.initializers.Constant(value= 0.1)

    self.W_T = self.add_weight(shape=(n_features, n_features),initializer = random_dist,trainable=True)
    self.b_T = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias, trainable=True)
    # self.W_T = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1),trainable=True)
    # self.b_T = tf.Variable(tf.constant(-1.0, shape=[n_data_points, n_sentences, n_features]),trainable=True)

    self.W = self.add_weight(shape=( n_features, n_features),initializer = random_dist, trainable=True)
    self.b = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias_2, trainable=True)
    # self.W = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1), trainable=True)
    # self.b = tf.Variable(tf.constant(0.1, shape=[n_data_points, n_sentences, n_features]), trainable=True)


  def call(self, inputs):
    x = inputs
    T = tf.sigmoid(tf.matmul(x, self.W_T) + self.b_T, name="transform_gate")
    H = tf.nn.relu(tf.matmul(x, self.W) + self.b, name="activation")
    C = tf.subtract(1.0, T, name="carry_gate")

    return tf.add(tf.multiply(H, T), tf.multiply(x, C), "y")

class text_audio_video:
# first input model
  input1 = Input(shape=(250,74), name = 'Audio_input')
  # highway1 = Highway()(input1)
  # highway5 = Highway()(highway1)
  # highway6 = Highway()(highway5)
  dense1 = Dense(74)(input1)

  # second input model
  input2 = Input(shape=(250,388), name = 'Video_input')
  # highway2 = Highway()(input2)
  # highway3 = Highway()(highway2)
  # highway4 = Highway()(highway3)
  dense7 = Dense(200)(input2)
  dense2 = Dense(74)(dense7)

  input3 = Input(shape = (250,5100), name = 'Text_input')
  dense4 = Dense(1000)(input3)
  dense5 = Dense(500)(dense4)
  dense6 = Dense(250)(dense5)
  dense3 = Dense(74)(dense6)
  # merge input models
  merge = concatenate([dense1,dense2,dense3], axis = 1)
  # interpretation model
  lstm = LSTM(128, dropout = 0.2, recurrent_dropout = 0.2)(merge)
  output = Dense(1, activation='sigmoid')(lstm)
  model = Model(inputs=[input1, input2, input3], outputs=output)
  # # summarize layers
  # print(model.summary())
  # # plot graph
  # plot_model(model)
  optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001)

  def run_model(self):
    self.model.compile(optimizer=self.optimizer, loss='binary_crossentropy')
    return self.model

# inp = np.array([audio_train,video_train,text_train], dtype = object)

In [10]:
#MAKE DATASET


dev_location = "dev_data"
test_location = "test_data"
train_location = "train_data"

devData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
testData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
trainData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]


dataset = np.concatenate((devData, np.concatenate((testData, trainData))))

gc.collect()
max_num_words = 17
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/MCA Dataset/GoogleNews-vectors-negative300.bin', binary=True)
stop_words = set(stopwords.words('english'))



def checkDataPointExistence(patientID, split):
  for i in split:
    if(patientID == i[0]):
      return True
  return False

def getData(patientID, location):
  # print("PatientID: " + str(int(patientID)))
  retData = [int(patientID)]
  textD = getTextData(patientID, location)
  audioD = getAudioData(patientID, location, textD)
  videoD = getVideoData(str(int(patientID)), location, textD)
  # patientD = np.concatenate((textD, audioD, videoD), axis = 1)
  # print("Final Patient Data: " + str(patientD.shape))
  return textD,audioD,videoD

def getTextData(patientID, location):
  fileName = "/content/drive/My Drive/MCA Dataset/"+ str(location) + "/" + str(int(patientID)) + "_TRANSCRIPT.csv"
  file = np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8', engine='python'))

  # Remove All Utterences By Ellie:
  for i in range(len(file)):
    if(file[i][2] != 'Participant'):
      np.delete(file, i)
      i-=1

  # Remove Speaker Columnn
  file = np.delete(file, 2, 1)

  # Convert Text Into Word Vectors:
  w2vs = np.zeros((1, max_num_words*300))
  for i in range(len(file)):
    sentence = file[i][2]
    w2v = returnWordToVec(sentence)
    w2vs = np.concatenate((w2vs, w2v), axis = 0)
  w2vs = np.delete(w2vs, 0, 0)

  # Delete Sentences and Replace With W2Vs
  file = np.delete(file, 2, 1)
  file = np.concatenate((file, w2vs), axis = 1)
  return file

def remove_StopWords(sentence):
    filtered_sentence = []
    for w in sentence:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

def returnWordToVec(sentence):
  global max_num_words, stop_words, model
  sentence = str(sentence).split(" ")
  sentence = remove_StopWords(sentence)
  index_word = 0
  wordMatrix = np.zeros(max_num_words*300)
  for j in range(min(max_num_words, len(sentence))):
    try:
      word = sentence[j]
      if(word[0] == '<'):
        if(word.find('>')!=-1):
          word = word[1:-1]
        else:
          word = word[1:]
      else:
        if(word.find('>')!=-1):
          word = word[0:-1]
      ss = np.array(model[word])
      wordMatrix[index_word*300:(index_word+1)*300] = ss
      index_word+=1
    except Exception as e:
      continue
  wordMatrix = np.array(wordMatrix.reshape(1,-1))
  return wordMatrix

def audioDataHelper(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    X = np.array(X)
    return X

def getAudioData(patientID, location, textD):
  fileName = "/content/drive/My Drive/MCA Dataset/"+ str(location) + "/" + str(int(patientID)) + "_COVAREP.csv"
  data = pd.read_csv(fileName,header = None)
  data = data.iloc[:,:].values
  data = audioDataHelper(data)
  # print("Audio Raw Data:" + str(data.shape))
  sentenceDatas = []
  for sentence in textD:
    sentenceStartime = sentence[0]
    sentenceEndTime = sentence[1]
    startIndex = math.floor(sentenceStartime/0.01)
    endIndex = math.ceil(sentenceEndTime/0.01)
    # print("Start Time: " + str(startIndex))
    # print("End Time: " + str(endIndex))
    sentenceData = data[startIndex: endIndex]
    sentenceData = np.average(sentenceData, axis = 0)
    # This might be a possible error
    sentenceData = np.array(sentenceData.reshape(1, -1))
    sentenceDatas.append(sentenceData)

  sentenceDatas = np.array(sentenceDatas)
  sentenceDatas = np.reshape(sentenceDatas, (textD.shape[0],-1))
  # print("Audio Final Data:" + str(sentenceDatas.shape))

  return sentenceDatas

def getVideoDataHelper(patientID, location):
  root = "/content/drive/My Drive/MCA Dataset/"+ str(location) + "/"
  file1 = root + (patientID)+"_CLNF_AUs.txt"
  file2 = root + (patientID)+"_CLNF_features.txt"
  file3 = root + (patientID)+"_CLNF_features3D.txt"
  file4 = root + (patientID)+"_CLNF_gaze.txt"
  file5 = root + (patientID)+"_CLNF_hog.txt"
  file6 = root + (patientID)+"_CLNF_pose.txt"
  data = processVideoData(file1)
  data = np.concatenate((data, processVideoData(file2)), 1)
  data = np.concatenate((data, processVideoData(file3)), 1)
  data = np.concatenate((data, processVideoData(file4)), 1)
  data = np.concatenate((data, processVideoData(file6)), 1)
  # print("Video Raw Data:" + str(data.shape))
  return data

def processVideoData(filename):
  try:
    data = pd.read_csv(filename,delimiter=',', dtype=float)
    X = data.iloc[:,:].values
    X = np.delete(X, 0, 1)
    X = np.delete(X, 1, 1)
  except:
    # print("Video Data corrupt, fixing.")
    data = pd.read_csv(filename,delimiter=',')
    X = data.iloc[:,:].values
    X = np.delete(X, 0, 1)
    X = np.delete(X, 1, 1)
    for i in range(len(X)):
        if(isinstance(X[i][5],str) or isinstance(X[i][7],str)):
            X[i] = np.zeros((1, X.shape[1]))
            # print("se" , end = " ")
  return X

def getVideoData(patientID, location, textD):
  data = getVideoDataHelper(patientID, location)
  sentenceDatas = []
  for sentence in textD:
    sentenceStartime = sentence[0]
    sentenceEndTime = sentence[1]
    startIndex = math.floor(sentenceStartime/0.333)
    endIndex = math.ceil(sentenceEndTime/0.333)
    # print("Start Time: " + str(startIndex))
    sentenceData = data[startIndex: endIndex]
    sentenceData = np.average(sentenceData, axis = 0)
    # This might be a possible error
    sentenceData = np.array(sentenceData.reshape(1, -1))
    sentenceDatas.append(sentenceData)

  sentenceDatas = np.array(sentenceDatas)
  sentenceDatas = np.reshape(sentenceDatas, (textD.shape[0],-1))
  # print("Video Final Data:" + str(sentenceDatas.shape))
  return sentenceDatas

# Xtrain = []
Ytrain = []
# Xtest = []
Ytest = []


audio_train = []
video_train = []
text_train = []

audio_test = []
video_test = []
text_test = []




for datapoint in dataset:
  # print(datapoint[0])
  if(checkDataPointExistence(datapoint[0], devData)):

    # Data Point in Dev Set
    text,audio,video = getData(datapoint[0], dev_location)
    audio_train.append(audio)
    video_train.append(video)
    text_train.append(text)
    # Xtest.append(data)
    Ytrain.append(datapoint[1])
    # print(data)
  elif(checkDataPointExistence(datapoint[0], testData)):
    # Data Point in Test Set
    text,audio,video = getData(datapoint[0], test_location)
    audio_test.append(audio)
    video_test.append(video)
    text_test.append(text)
    # Xtest.append(data)
    Ytest.append(datapoint[1])
  elif(checkDataPointExistence(datapoint[0], trainData)):
    # Data Point in Train Set
    text,audio,video = getData(datapoint[0], train_location)
    audio_train.append(audio)
    video_train.append(video)
    text_train.append(text)
    # Xtest.append(data)
    Ytrain.append(datapoint[1])

def refactor(arr, size):
  arrsize = arr.shape[0]
  temp = np.zeros((size, arr.shape[1]))
  for i in range(min(len(arr), size)):
    temp[i] = arr[i]
  return temp

numberOfSentences = 250

devData = []
trainData = []
testData = []
gc.collect()

for i in range(len(audio_train)):
  audio_train[i] = refactor(audio_train[i], numberOfSentences)
  video_train[i] = refactor(video_train[i], numberOfSentences)
  text_train[i] = refactor(text_train[i], numberOfSentences)
  # print(Xtrain[i].shape)

# for i in range(len(audio_dev)):
#   audio_dev[i] = refactor(audio_dev[i], numberOfSentences)
#   video_dev[i] = refactor(video_dev[i], numberOfSentences)
#   text_dev[i] = refactor(text_dev[i], numberOfSentences)
#   # print(Xtrain[i].shape)

for i in range(len(audio_test)):
  audio_test[i] = refactor(audio_train[i], numberOfSentences)
  video_test[i] = refactor(video_train[i], numberOfSentences)
  text_test[i] = refactor(text_train[i], numberOfSentences)
  # print(Xtest[i].shape)
audio_test = np.array(audio_test)
video_test = np.array(video_test)
text_test = np.array(text_test)
text_test = text_test[:,:,2:]

audio_train = np.array(audio_train)
video_train = np.array(video_train)
text_train = np.array(text_train)
text_train = text_train[:,:,2:]

# audio_dev = np.array(audio_dev)
# video_dev = np.array(video_dev)
# text_dev = np.array(text_dev)
# text_dev = text_dev[:,:,2:]

dataset = []
gc.collect()

print(audio_test.shape,video_test.shape,text_test.shape)
print(audio_train.shape,video_train.shape,text_train.shape)
# print(audio_dev.shape,video_dev.shape,text_dev.shape)

Ytrain = np.array(Ytrain)
Ytest = np.array(Ytest)


import sklearn
from sklearn import preprocessing


def upsample(X_train,Y_train):
  X_train_0 = X_train[Y_train==0]
  X_train_1 = X_train[Y_train==1]

  Y_train_1 = Y_train[Y_train==1]
  # print(Y_train_1.shape)
  # print(X_train_1.shape)
  size = X_train_0.shape[0] - X_train_1.shape[0]
  X = []
  Y = []
  X_train = list(X_train)
  Y_train = list(Y_train)
  while(size>0):
    size -= 1
    index = np.random.randint(0,X_train_1.shape[0]-1)
    leave_index = np.random.randint(0,len(X_train)-1)
    X_add = X_train_1[index]
    X_leave = X_train[leave_index]

    Y_add = Y_train_1[index]
    Y_leave = Y_train[leave_index]

    X_train[leave_index] = X_add
    X_train.append(X_leave)

    Y_train[leave_index] = Y_add
    Y_train.append(Y_leave)


  X_train = np.array(X_train)
  Y_train = np.array(Y_train)
  return X_train,Y_train




audio_train = np.nan_to_num(audio_train)
video_train = np.nan_to_num(video_train)
text_train = np.nan_to_num(text_train)

audio_train, _ = upsample(audio_train,Ytrain)
video_train, _ = upsample(video_train,Ytrain)
text_train, Ytrain = upsample(text_train,Ytrain)

print(audio_train.shape)
print(video_train.shape)
print(text_train.shape)
print(Ytrain.shape)

for i in range(audio_train.shape[0]):
  audio_train[i] = sklearn.preprocessing.normalize(audio_train[i])
  video_train[i] = sklearn.preprocessing.normalize(video_train[i])
  text_train[i] = sklearn.preprocessing.normalize(text_train[i])


print(Ytest.shape)



audio_test = np.nan_to_num(audio_test)
video_test = np.nan_to_num(video_test)
text_test = np.nan_to_num(text_test)


for i in range(audio_test.shape[0]):
  audio_test[i] = sklearn.preprocessing.normalize(audio_test[i])
  video_test[i] = sklearn.preprocessing.normalize(video_test[i])
  text_test[i] = sklearn.preprocessing.normalize(text_test[i])


<ipython-input-10-567983f66a27>:155: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filename,delimiter=',')
<ipython-input-10-567983f66a27>:155: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,

(47, 250, 74) (47, 250, 388) (47, 250, 5100)
(142, 250, 74) (142, 250, 388) (142, 250, 5100)
(200, 250, 74)
(200, 250, 388)
(200, 250, 5100)
(200,)
(47,)


In [11]:

def Thresholding(Y_pred, threshold):
  Y_pred2 = []
  print("Y_pred: ", Y_pred.shape)
  for i in range(len(Y_pred)):
    if(Y_pred[i] < threshold):
      Y_pred2.append(0)
    else:
      Y_pred2.append(1)

  return np.array(Y_pred2)

In [13]:
# Define the model class
class TextModel:
    def __init__(self):
        input3 = Input(shape=(250, 5100))
        dense4 = Dense(1000)(input3)
        dense5 = Dense(500)(dense4)
        dense6 = Dense(250)(dense5)

        lstm = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(dense6)
        output = Dense(1, activation='sigmoid')(lstm)

        self.model = Model(inputs=input3, outputs=output)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

    def run_model(self):
        self.model.compile(optimizer=self.optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        return self.model

# Run and train the model
print("-------------------------------------- TEXT (W/o GATING) SENTENCE LEVEL-------------------------------------------------")
model1 = TextModel()
model = model1.run_model()

model.fit(
    text_train, Ytrain,
    epochs=50,
    validation_split=0.2,
    callbacks=tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=20, restore_best_weights=True
    ),
    batch_size=137
)

# Evaluate
pred = model.predict(text_test)
print(classification_report(Ytest, Thresholding(pred, 0.5)))

-------------------------------------- TEXT (W/o GATING) SENTENCE LEVEL-------------------------------------------------
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.4550 - loss: 0.6930 - val_accuracy: 0.6250 - val_loss: 0.6923
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.6606 - loss: 0.6892 - val_accuracy: 0.6000 - val_loss: 0.6903
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.6467 - loss: 0.6861 - val_accuracy: 0.6000 - val_loss: 0.6881
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7051 - loss: 0.6823 - val_accuracy: 0.6250 - val_loss: 0.6854
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7020 - loss: 0.6784 - val_accuracy: 0.6000 - val_loss: 0.6814
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7037 - loss: 0.6747 - val_accuracy: 0.6500 - val_loss: 0.6801
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7117 - loss: 0.6692 - val_accuracy: 0.6000 - val_loss: 0.6813
Epoch 8/50
2/2 ━━

In [14]:
class AudioModel:
    def __init__(self):
        input_layer = Input(shape=(250, 74))  # Adjusted to match audio input shape
        lstm_layer = LSTM(60, dropout=0.2, recurrent_dropout=0.2)(input_layer)
        output_layer = Dense(1, activation='sigmoid')(lstm_layer)

        self.model = Model(inputs=input_layer, outputs=output_layer)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

    def run_model(self):
        self.model.compile(optimizer=self.optimizer, loss='binary_crossentropy')
        return self.model

# -------------------------- Training & Evaluation -------------------------
print("-------------------------------------- AUDIO (W/o GATING) SENTENCE LEVEL -------------------------------------------------")

# Instantiate and compile model
model1 = AudioModel()
model = model1.run_model()

# Train the model
model.fit(
    audio_train, Ytrain,
    validation_split=0.2,
    callbacks=tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=30,
        verbose=0,
        mode='min',
        restore_best_weights=True
    ),
    epochs=30,
    batch_size=137
)

# Predict on test set
pred = model.predict(audio_test)

# Evaluate performance using classification report
print(classification_report(Ytest, Thresholding(pred, 0.5)))

-------------------------------------- AUDIO (W/o GATING) SENTENCE LEVEL -------------------------------------------------
Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 562ms/step - loss: 0.6956 - val_loss: 0.6882
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - loss: 0.6976 - val_loss: 0.6883
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - loss: 0.6949 - val_loss: 0.6883
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - loss: 0.6997 - val_loss: 0.6884
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - loss: 0.7011 - val_loss: 0.6884
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - loss: 0.6912 - val_loss: 0.6885
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - loss: 0.6945 - val_loss: 0.6885
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step - loss: 0.6952 - val_loss: 0.6886
Epoch 9/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - loss: 0.7012 - val_loss: 0.6886
Epoch 10/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - loss: 0.6935 - val_loss: 0.6886
Epoch 11/30
2/2 ━━━━━━━━━━━━━━━━━━━━

In [15]:

# -------------------------- Video Model Class ------------------------------
class VideoModel:
    def __init__(self):
        input_layer = Input(shape=(250, 388))  # Input shape for video features
        dense_layer = Dense(250)(input_layer)  # Optional feature transformation
        lstm_layer = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(dense_layer)
        output_layer = Dense(1, activation='sigmoid')(lstm_layer)

        self.model = Model(inputs=input_layer, outputs=output_layer)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

        # Optional model summary and architecture visualization
        print(self.model.summary())
        # plot_model(self.model, to_file='video_model.png')

    def run_model(self):
        self.model.compile(optimizer=self.optimizer, loss='binary_crossentropy')
        return self.model

# -------------------------- Training & Evaluation --------------------------
print("-------------------------------------- VIDEO (W/o GATING) SENTENCE LEVEL -------------------------------------------------")

# Instantiate and compile model
model1 = VideoModel()
model = model1.run_model()

# Train the model
model.fit(
    video_train, Ytrain,
    validation_split=0.2,
    callbacks=tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=30,
        verbose=0,
        mode='min',
        restore_best_weights=True
    ),
    epochs=30,
    batch_size=137
)

# Predict on test set
pred = model.predict(video_test)

# Evaluate performance using classification report
print(classification_report(Ytest, Thresholding(pred, 0.5)))

-------------------------------------- VIDEO (W/o GATING) SENTENCE LEVEL -------------------------------------------------


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 250, 388)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 250, 250)       │        97,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 128)            │       194,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 291,427 (1.11 MB)

 Trainable params: 291,427 (1.11 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 724ms/step - loss: 0.6960 - val_loss: 0.6953
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 289ms/step - loss: 0.6932 - val_loss: 0.6920
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 298ms/step - loss: 0.6942 - val_loss: 0.6892
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 294ms/step - loss: 0.6930 - val_loss: 0.6868
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 305ms/step - loss: 0.6933 - val_loss: 0.6856
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 310ms/step - loss: 0.6939 - val_loss: 0.6857
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 326ms/step - loss: 0.6933 - val_loss: 0.6863
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 303ms/step - loss: 0.6931 - val_loss: 0.6867
Epoch 9/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 297ms/step - loss: 0.6918 - val_loss: 0.6870
Epoch 10/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 308ms/step - loss: 0.6921 - val_loss: 0.6866
Epoch 11/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 299ms/step - loss: 0.6926 - val_loss: 0.6855
Epoch 12/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 311ms/step - loss: 0.6910 - v

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 299ms/step
Y_pred:  (47, 1)
              precision    recall  f1-score   support

         0.0       0.84      0.64      0.72        33
         1.0       0.45      0.71      0.56        14

    accuracy                           0.66        47
   macro avg       0.65      0.68      0.64        47
weighted avg       0.73      0.66      0.67        47



In [16]:
# -------------------------- Audio + Text Model Class ------------------------------
class TextAudioModel:
    def __init__(self):
        # Audio Input
        input1 = Input(shape=(250, 74), name='Audio_input')
        dense1 = Dense(74)(input1)  # Audio processing

        # Text Input
        input3 = Input(shape=(250, 5100), name='Text_input')
        dense4 = Dense(1000)(input3)
        dense5 = Dense(500)(dense4)
        dense6 = Dense(250)(dense5)
        dense3 = Dense(74)(dense6)  # Text processing

        # Merge Audio and Text features
        merge = concatenate([dense1, dense3], axis=1)

        # LSTM for sequential interpretation
        lstm = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(merge)

        # Output layer
        output = Dense(1, activation='sigmoid')(lstm)

        # Model Definition
        self.model = Model(inputs=[input1, input3], outputs=output)

        # Optimizer
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

        # Optional model summary and visualization
        print(self.model.summary())
        # plot_model(self.model, to_file='audio_text_model.png')

    def run_model(self):
        self.model.compile(optimizer=self.optimizer, loss='binary_crossentropy')
        return self.model

# -------------------------- Training & Evaluation --------------------------
print("-------------------------------------- AUDIO + TEXT (W/o GATING) SENTENCE LEVEL -------------------------------------------------")

# Instantiate and compile model
model1 = TextAudioModel()
model = model1.run_model()

# Train the model
model.fit(
    [audio_train, text_train], Ytrain,
    validation_split=0.2,
    callbacks=tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=15,
        verbose=0,
        mode='min',
        restore_best_weights=True
    ),
    epochs=30,
    batch_size=137
)

# Predict on test set
pred = model.predict([audio_test, text_test])
pred2 = model.predict([audio_train, text_train])

# Evaluate performance using classification report
print(classification_report(Ytest, Thresholding(pred, 0.5)))
print("TRAINING ACCURACY: ", classification_report(Ytrain, Thresholding(pred2, 0.5)))
print(classification_report(Ytest, Thresholding(pred, 0.6)))
print(classification_report(Ytest, Thresholding(pred, 0.4)))
print(classification_report(Ytest, Thresholding(pred, 0.3)))
print(classification_report(Ytest, Thresholding(pred, 0.7)))

-------------------------------------- AUDIO + TEXT (W/o GATING) SENTENCE LEVEL -------------------------------------------------


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Text_input          │ (None, 250, 5100) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_34 (Dense)    │ (None, 250, 1000) │  5,101,000 │ Text_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_35 (Dense)    │ (None, 250, 500)  │    500,500 │ dense_34[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Audio_input         │ (None, 250, 74)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_36 (Dense)    │ (None, 250, 250)  │    125,250 │ dense_35[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_33 (Dense)    │ (None, 250, 74)   │      5,550 │ Audio_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_37 (Dense)    │ (None, 250, 74)   │     18,574 │ dense_36[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 500, 74)   │          0 │ dense_33[0][0],   │
│ (Concatenate)       │                   │            │ dense_37[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_9 (LSTM)       │ (None, 128)       │    103,936 │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_38 (Dense)    │ (None, 1)         │        129 │ lstm_9[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,854,939 (22.33 MB)

 Trainable params: 5,854,939 (22.33 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - loss: 0.6929 - val_loss: 0.6933
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6902 - val_loss: 0.6927
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.6881 - val_loss: 0.6926
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6851 - val_loss: 0.6918
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.6824 - val_loss: 0.6917
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6798 - val_loss: 0.6909
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.6742 - val_loss: 0.6891
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6702 - val_loss: 0.6867
Epoch 9/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.6646 - val_loss: 0.6849
Epoch 10/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6554 - val_loss: 0.6800
Epoch 11/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.6485 - val_loss: 0.6697
Epoch 12/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6394 - val_loss: 0.6586
Epoch 13/30
2/2 ━━━

In [17]:
# -------------------------- Video + Text Model Class ------------------------------
class TextVideoModel:
    def __init__(self):
        # Video Input
        input2 = Input(shape=(250, 388), name='Video_input')
        dense2 = Dense(250)(input2)  # Video processing

        # Text Input
        input3 = Input(shape=(250, 5100), name='Text_input')
        dense4 = Dense(1000)(input3)
        dense5 = Dense(500)(dense4)
        dense3 = Dense(250)(dense5)  # Text processing

        # Merge Video and Text features
        merge = concatenate([dense2, dense3], axis=1)

        # LSTM for sequential interpretation
        lstm = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(merge)

        # Output layer
        output = Dense(1, activation='sigmoid')(lstm)

        # Model Definition
        self.model = Model(inputs=[input2, input3], outputs=output)

        # Optimizer
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

        # Optional model summary and visualization
        print(self.model.summary())
        # plot_model(self.model, to_file='video_text_model.png')

    def run_model(self):
        self.model.compile(optimizer=self.optimizer, loss='binary_crossentropy')
        return self.model

# -------------------------- Training & Evaluation --------------------------
print("-------------------------------------- VIDEO + TEXT (W/O GATING) SENTENCE LEVEL -------------------------------------------------")

# Instantiate and compile model
model1 = TextVideoModel()
model = model1.run_model()

# Train the model
model.fit(
    [video_train, text_train], Ytrain,
    validation_split=0.2,
    callbacks=tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=15,
        verbose=0,
        mode='min',
        restore_best_weights=True
    ),
    epochs=30,
    batch_size=137
)

# Predict on test set
pred = model.predict([video_test, text_test])
pred2 = model.predict([video_train, text_train])

# Evaluate performance using classification report
print(classification_report(Ytest, Thresholding(pred, 0.5)))
print("TRAINING ACCURACY: ", classification_report(Ytrain, Thresholding(pred2, 0.5)))
print(classification_report(Ytest, Thresholding(pred, 0.6)))
print(classification_report(Ytest, Thresholding(pred, 0.4)))
print(classification_report(Ytest, Thresholding(pred, 0.3)))
print(classification_report(Ytest, Thresholding(pred, 0.7)))

-------------------------------------- VIDEO + TEXT (W/O GATING) SENTENCE LEVEL -------------------------------------------------


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Text_input          │ (None, 250, 5100) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_40 (Dense)    │ (None, 250, 1000) │  5,101,000 │ Text_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Video_input         │ (None, 250, 388)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_41 (Dense)    │ (None, 250, 500)  │    500,500 │ dense_40[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_39 (Dense)    │ (None, 250, 250)  │     97,250 │ Video_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_42 (Dense)    │ (None, 250, 250)  │    125,250 │ dense_41[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 500, 250)  │          0 │ dense_39[0][0],   │
│ (Concatenate)       │                   │            │ dense_42[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_10 (LSTM)      │ (None, 128)       │    194,048 │ concatenate_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_43 (Dense)    │ (None, 1)         │        129 │ lstm_10[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,018,177 (22.96 MB)

 Trainable params: 6,018,177 (22.96 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 0.6929 - val_loss: 0.6939
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6886 - val_loss: 0.6931
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6846 - val_loss: 0.6888
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6806 - val_loss: 0.6851
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6757 - val_loss: 0.6823
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6700 - val_loss: 0.6796
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - loss: 0.6652 - val_loss: 0.6756
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6584 - val_loss: 0.6701
Epoch 9/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6508 - val_loss: 0.6666
Epoch 10/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6422 - val_loss: 0.6674
Epoch 11/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6338 - val_loss: 0.6689
Epoch 12/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6196 - val_loss: 0.6673
Epoch 13/30
2/2 ━━━

In [19]:
# -------------------------- Audio + Text + Video Model Class ------------------------------
class TextAudioVideoModel:
    def __init__(self):
        # Audio Input
        input1 = Input(shape=(250, 74), name='Audio_input')
        dense1 = Dense(74)(input1)  # Audio processing

        # Video Input
        input2 = Input(shape=(250, 388), name='Video_input')
        dense7 = Dense(200)(input2)
        dense2 = Dense(74)(dense7)  # Video processing

        # Text Input
        input3 = Input(shape=(250, 5100), name='Text_input')
        dense4 = Dense(1000)(input3)
        dense5 = Dense(500)(dense4)
        dense6 = Dense(250)(dense5)
        dense3 = Dense(74)(dense6)  # Text processing

        # Merge Audio, Video, and Text features
        merge = concatenate([dense1, dense2, dense3], axis=1)

        # LSTM for sequential interpretation
        lstm = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(merge)

        # Output layer
        output = Dense(1, activation='sigmoid')(lstm)

        # Model Definition
        self.model = Model(inputs=[input1, input2, input3], outputs=output)

        # Optimizer
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

        # Optional model summary and visualization
        print(self.model.summary())
        # plot_model(self.model, to_file='audio_text_video_model.png')

    def run_model(self):
        self.model.compile(optimizer=self.optimizer, loss='binary_crossentropy')
        return self.model

# -------------------------- Training & Evaluation --------------------------
print("-------------------------------------- AUDIO + TEXT + VIDEO (W/O GATING) SENTENCE LEVEL -------------------------------------------------")

# Instantiate and compile model
model1 = TextAudioVideoModel()
model = model1.run_model()

# Train the model
model.fit(
    [audio_train, video_train, text_train], Ytrain,
    validation_split=0.2,
    callbacks=tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=15,
        verbose=0,
        mode='min',
        restore_best_weights=True
    ),
    epochs=30,
    batch_size=137
)

# Predict on test set
pred = model.predict([audio_test, video_test, text_test])
pred2 = model.predict([audio_train, video_train, text_train])

# Evaluate performance using classification report
print(classification_report(Ytest, Thresholding(pred, 0.5)))
print("TRAINING ACCURACY: ", classification_report(Ytrain, Thresholding(pred2, 0.5)))
print(classification_report(Ytest, Thresholding(pred, 0.6)))
print(classification_report(Ytest, Thresholding(pred, 0.4)))
print(classification_report(Ytest, Thresholding(pred, 0.3)))
print(classification_report(Ytest, Thresholding(pred, 0.7)))

-------------------------------------- AUDIO + TEXT + VIDEO (W/O GATING) SENTENCE LEVEL -------------------------------------------------


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Text_input          │ (None, 250, 5100) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_55 (Dense)    │ (None, 250, 1000) │  5,101,000 │ Text_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Video_input         │ (None, 250, 388)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_56 (Dense)    │ (None, 250, 500)  │    500,500 │ dense_55[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Audio_input         │ (None, 250, 74)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_53 (Dense)    │ (None, 250, 200)  │     77,800 │ Video_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_57 (Dense)    │ (None, 250, 250)  │    125,250 │ dense_56[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_52 (Dense)    │ (None, 250, 74)   │      5,550 │ Audio_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_54 (Dense)    │ (None, 250, 74)   │     14,874 │ dense_53[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_58 (Dense)    │ (None, 250, 74)   │     18,574 │ dense_57[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, 750, 74)   │          0 │ dense_52[0][0],   │
│ (Concatenate)       │                   │            │ dense_54[0][0],   │
│                     │                   │            │ dense_58[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_12 (LSTM)      │ (None, 128)       │    103,936 │ concatenate_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_59 (Dense)    │ (None, 1)         │        129 │ lstm_12[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,947,613 (22.69 MB)

 Trainable params: 5,947,613 (22.69 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - loss: 0.6933 - val_loss: 0.6912
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - loss: 0.6898 - val_loss: 0.6883
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6870 - val_loss: 0.6868
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6839 - val_loss: 0.6859
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6799 - val_loss: 0.6842
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6773 - val_loss: 0.6818
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6706 - val_loss: 0.6787
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6675 - val_loss: 0.6752
Epoch 9/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6595 - val_loss: 0.6710
Epoch 10/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6504 - val_loss: 0.6685
Epoch 11/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6420 - val_loss: 0.6696
Epoch 12/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6333 - val_loss: 0.6730
Epoch 13/30
2/2 ━━━